In [4]:
import pandas as pd
import numpy as np

data = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
sub = pd.read_csv('sample_submission.csv')

In [5]:
from stop_words import get_stop_words
# 불용어 리스트
languages = [
   'English'
]
my_stop_word_list = []
for i in languages:
    my_stop_word_list.append(get_stop_words(i.lower()))
my_stop_word_list = sum(my_stop_word_list, [])
my_stop_word_list = my_stop_word_list + ['paperbook', 'hardcover']

In [6]:
# https://gist.github.com/nealrs/96342d8231b75cf4bb82
contractions = {
  "ain't": "am not",
  "aren't": "are not",
  "can't": "cannot",
  "can't've": "cannot have",
  "'cause": "because",
  "could've": "could have",
  "couldn't": "could not",
  "couldn't've": "could not have",
  "didn't": "did not",
  "doesn't": "does not",
  "don't": "do not",
  "hadn't": "had not",
  "hadn't've": "had not have",
  "hasn't": "has not",
  "haven't": "have not",
  "he'd": "he would",
  "he'd've": "he would have",
  "he'll": "he will",
  "he'll've": "he will have",
  "he's": "he is",
  "how'd": "how did",
  "how'd'y": "how do you",
  "how'll": "how will",
  "how's": "how is",
  "I'd": "I would",
  "I'd've": "I would have",
  "I'll": "I will",
  "I'll've": "I will have",
  "I'm": "I am",
  "I've": "I have",
  "isn't": "is not",
  "it'd": "it had",
  "it'd've": "it would have",
  "it'll": "it will",
  "it'll've": "it will have",
  "it's": "it is",
  "let's": "let us",
  "ma'am": "madam",
  "mayn't": "may not",
  "might've": "might have",
  "mightn't": "might not",
  "mightn't've": "might not have",
  "must've": "must have",
  "mustn't": "must not",
  "mustn't've": "must not have",
  "needn't": "need not",
  "needn't've": "need not have",
  "o'clock": "of the clock",
  "oughtn't": "ought not",
  "oughtn't've": "ought not have",
  "shan't": "shall not",
  "sha'n't": "shall not",
  "shan't've": "shall not have",
  "she'd": "she would",
  "she'd've": "she would have",
  "she'll": "she will",
  "she'll've": "she will have",
  "she's": "she is",
  "should've": "should have",
  "shouldn't": "should not",
  "shouldn't've": "should not have",
  "so've": "so have",
  "so's": "so is",
  "that'd": "that would",
  "that'd've": "that would have",
  "that's": "that is",
  "there'd": "there had",
  "there'd've": "there would have",
  "there's": "there is",
  "they'd": "they would",
  "they'd've": "they would have",
  "they'll": "they will",
  "they'll've": "they will have",
  "they're": "they are",
  "they've": "they have",
  "to've": "to have",
  "wasn't": "was not",
  "we'd": "we had",
  "we'd've": "we would have",
  "we'll": "we will",
  "we'll've": "we will have",
  "we're": "we are",
  "we've": "we have",
  "weren't": "were not",
  "what'll": "what will",
  "what'll've": "what will have",
  "what're": "what are",
  "what's": "what is",
  "what've": "what have",
  "when's": "when is",
  "when've": "when have",
  "where'd": "where did",
  "where's": "where is",
  "where've": "where have",
  "who'll": "who will",
  "who'll've": "who will have",
  "who's": "who is",
  "who've": "who have",
  "why's": "why is",
  "why've": "why have",
  "will've": "will have",
  "won't": "will not",
  "won't've": "will not have",
  "would've": "would have",
  "wouldn't": "would not",
  "wouldn't've": "would not have",
  "y'all": "you all",
  "y'alls": "you alls",
  "y'all'd": "you all would",
  "y'all'd've": "you all would have",
  "y'all're": "you all are",
  "y'all've": "you all have",
  "you'd": "you had",
  "you'd've": "you would have",
  "you'll": "you you will",
  "you'll've": "you you will have",
  "you're": "you are",
  "you've": "you have"
}


In [7]:
import re

#main 단어만 추출 > 차후 이 단어들로 test
def preprocessing_sentence(sentence, remove_stopwords = True):
    #소문자화
    sentence = sentence.lower()
    #괄호로 닫친 문자열 괄호 제거
    sentence = re.sub(r'\([^)]*\)',' ',sentence)
    #쌍따옴표 제거
    sentence = re.sub('"', ' ', sentence) 
    #약어 정규화
    sentence = ' '.join([contractions[t] if t in contractions else t for t in sentence.split(" ")])
    #소유격 제거
    sentence = re.sub(r"'s\b"," ",sentence)
    #특수문자 제거
    sentence = re.sub("[^a-zA-Z]"," ", sentence)
    
    #불용어 제거
    if my_stop_word_list:
        tokens = ' '.join(word for word in sentence.split() if not word in my_stop_word_list if len(word)>1)
    else:
        tokens = ' '.join(word for word in sentence.split() if len(word)>1)
    return tokens
data['facts'] = [preprocessing_sentence(i) for i in data['facts']]
test['facts'] = [preprocessing_sentence(i) for i in test['facts']]

data['facts'] = data['facts'].astype('str')
test['facts'] = test['facts'].astype('str')

facts_corpus_lose = ' '.join(data[data['first_party_winner'] == 0]['facts'])
facts_corpus_win = ' '.join(data[data['first_party_winner'] == 1]['facts'])

In [8]:
import pandas as pd
from collections import Counter
import nltk
nltk.download('punkt')


# 데이터프레임에서 문장 추출
sentences = data['facts'].tolist()

# 문장을 단어로 토큰화
tokenized_sentences = [nltk.word_tokenize(sentence) for sentence in sentences]

# 모든 문장에서의 단어 빈도수 합산
word_frequencies = Counter()
for sentence in tokenized_sentences:
    word_frequencies.update(sentence)

# 1000개 이하의 빈도수를 가진 단어 제거하여 문장 재구성
filtered_sentences = []
for sentence in tokenized_sentences:
    filtered_sentence = ' '.join([word for word in sentence if word_frequencies[word] > 300])
    filtered_sentences.append(filtered_sentence)

# 필터링된 문장들을 데이터프레임 열에 할당
data['filtered_text'] = filtered_sentences

# 결과 확인
data

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\kweon\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


,ID,first_party,second_party,facts,first_party_winner,filtered_text
0,TRAIN_0000,Phil A. St. Amant,Herman A. Thompson,june phil st amant candidate public office mad...,1,public sued first circuit court appeals revers...
1,TRAIN_0001,Stephen Duncan,Lawrence Owens,ramon nelson riding bike suffered lethal blow ...,0,two convicted death judge trial ruled also fou...
2,TRAIN_0002,Billy Joe Magwood,"Tony Patterson, Warden, et al.",alabama state court convicted billy joe magwoo...,1,state court convicted murder sentenced death f...
3,TRAIN_0003,Linkletter,Walker,victor linkletter convicted state court eviden...,0,convicted state court evidence police supreme ...
4,TRAIN_0004,William Earl Fikes,Alabama,april selma alabama intruder broke apartment d...,1,city several trial police held police later ti...
...,...,...,...,...,...,...
2473,TRAIN_2473,"HollyFrontier Cheyenne Refining, LLC, et al.","Renewable Fuels Association, et al.",congress amended clean air act energy policy a...,1,act act several one year three alleging circui...
2474,TRAIN_2474,"Grupo Mexicano de Desarrollo, S. A.","Alliance Bond Fund, Inc.",alliance bond fund inc investment fund purchas...,1,holding company government filed suit federal ...
2475,TRAIN_2475,Peguero,United States,district court sentenced manuel peguero months...,0,district court sentenced guilty federal court ...
2476,TRAIN_2476,Immigration and Naturalization Service,St. Cyr,march enrico st cyr lawful permanent resident ...,0,guilty court conviction death act act section ...


In [9]:
data['first_party'].value_counts()

United States       154
Illinois              9
Maryland              8
Florida               8
New York              7
                   ... 
David Carpenter       1
Larry Gene Heath      1
PGA TOUR, Inc.        1
PPL Montana, LLC      1
Markman               1
Name: first_party, Length: 2110, dtype: int64

In [10]:
data['second_party'].value_counts()


United States                        240
California                            19
United States of America              15
Illinois                              13
Federal Communications Commission     10
                                    ... 
David Boren, Governor of Oklahoma      1
Federal Bureau of Prisons et al.       1
Town of Harrison                       1
Charles Burr et al.                    1
Westview Instruments, Inc.             1
Name: second_party, Length: 1974, dtype: int64

# Train

In [11]:
from sklearn.utils import resample

subset_0 = data[data["first_party_winner"] == 0]
subset_1 = data[data["first_party_winner"] == 1]

subset_1_downsampled = resample(subset_1,
                                replace=False,
                                n_samples= 829,
                                random_state=42)

data = pd.concat([subset_0, subset_1_downsampled])

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
def get_vector(vectorizer, df, train_mode):
    if train_mode:
        X_filtered_text = vectorizer.fit_transform(df['filtered_text'])
    else:
        X_filtered_text = vectorizer.transform(df['facts'])
    X_party1 = vectorizer.transform(df['first_party'])
    X_party2 = vectorizer.transform(df['second_party'])
    
    X = np.concatenate([X_party1.todense(), X_party2.todense(), X_filtered_text.todense()], axis=1)
    return X

In [13]:
X = get_vector(vectorizer, data, True)
Y = data["first_party_winner"]
X_T = get_vector(vectorizer, test, False)

X = np.squeeze(np.asarray(X))
X_T = np.squeeze(np.asarray(X_T))

In [14]:
X_T.shape
X.shape

(1658, 294)

In [15]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler(feature_range=(0,3))

scaler.fit(X)
X_scaled = scaler.transform(X)
X_T_scaled = scaler.transform(X_T)

In [16]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

X_train, X_test, y_train, y_test = train_test_split(X_scaled, Y, test_size=0.2, random_state=42 , stratify=Y)

In [17]:
cat_params = {
                'verbose' : 0,
                'random_state': 113,
               # 'use_best_model' : True,
             }

In [18]:
from sklearn import svm
# Classifiers
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import RidgeClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import AdaBoostClassifier
from catboost import CatBoostClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier


# Classifiers
names = [
    "Logistic Regression",
    "KNN Classifier",
    "Decision Tree",
    "LinearSVC",
    "Linear SVM",
    "Poly SVM",
    "RBF SVM",
    "sigmoid SVM",
    "Neural Network",
    "Random Forest",
    "SGD Classifier",
    "Ridge Classifier",
    "XGBoost",
    "AdaBoost",
    "Catboost",
    "Gaussian Naive Bayes"
]

models = [
    LogisticRegression(max_iter = 1000),
    KNeighborsClassifier(n_neighbors = 149, n_jobs = -1),
    DecisionTreeClassifier(),
    LinearSVC(),
    svm.SVC(kernel = 'linear'),
    svm.SVC(kernel = 'poly'),
    svm.SVC(kernel = 'rbf'),
    svm.SVC(kernel = 'sigmoid'),
    MLPClassifier(max_iter=300),
    RandomForestClassifier(n_estimators = 100),
    SGDClassifier(loss = 'hinge'),
    RidgeClassifier(),
    XGBClassifier(),
    AdaBoostClassifier(),
    CatBoostClassifier(**cat_params),
    GaussianNB()
]

In [21]:
import time
# Function to return summary of baseline models
def score(X_train, y_train, X_val, y_val, names = names, models = models):
    score_df, score_train, score_val = pd.DataFrame(), [], []
    x = time.time()
    for model in models:
        model.fit(X_train, y_train)
        y_train_pred, y_val_pred = model.predict(X_train), model.predict(X_val)
        score_train.append(accuracy_score(y_train, y_train_pred))
        score_val.append(accuracy_score(y_val, y_val_pred))
    
    score_df["Classifier"], score_df["Training accuracy"], score_df["Validation accuracy"] = names, score_train, score_val
    score_df.sort_values(by = 'Validation accuracy', ascending = False, inplace = True)
    return score_df

In [22]:
score(X_train, y_train, X_test, y_test, names = names, models = models)

,Classifier,Training accuracy,Validation accuracy
5,Poly SVM,0.952489,0.608434
14,Catboost,0.963801,0.575301
6,RBF SVM,0.838612,0.557229
12,XGBoost,0.996229,0.557229
9,Random Forest,0.999246,0.548193
13,AdaBoost,0.700603,0.548193
8,Neural Network,0.999246,0.545181
7,sigmoid SVM,0.567119,0.539157
1,KNN Classifier,0.608597,0.533133
11,Ridge Classifier,0.668929,0.527108


# predict

In [44]:
def pred_score(X, Y, X_T, names, models):
    score_df = pd.DataFrame()
    pred_0, pred_1 = [], []
    for model in models:
        model.fit(X, Y)
        pred = model.predict(X_T)
        pred_0.append(pd.DataFrame(pred).value_counts().to_frame().T[0].values[0][0])
        pred_1.append(pd.DataFrame(pred).value_counts().to_frame().T[1].values[0][0])
    score_df["Classifier"], score_df["0"], score_df["1"] = names, pred_0, pred_1
    return score_df    

pred_score(X=X_scaled, Y=Y, X_T=X_T_scaled, names=names, models=models)

c:\Users\kweon\anaconda3\envs\gibo\lib\site-packages\sklearn\svm\_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


,Classifier,0,1
0,Logistic Regression,563,677
1,KNN Classifier,538,702
2,Decision Tree,603,637
3,LinearSVC,558,682
4,Linear SVM,574,666
5,Poly SVM,570,670
6,RBF SVM,558,682
7,sigmoid SVM,554,686
8,Neural Network,625,615
9,Random Forest,599,641


In [26]:
# model = KNeighborsClassifier(n_neighbors = 149, n_jobs = -1)
# model = svm.SVC(kernel = 'poly')
model = svm.SVC(kernel = 'sigmoid')
model.fit(X_scaled, Y)
pred = model.predict(X_T_scaled)

In [43]:
pd.DataFrame(pred).value_counts().to_frame().T[1].values[0][0]

686

In [27]:
sub['first_party_winner']= pred
sub['first_party_winner'].value_counts()

1    686
0    554
Name: first_party_winner, dtype: int64

In [25]:
sub.to_csv('del300_downsampling_scaling_svmpoly_230626.csv', index=False)